# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Read csv file

output_data_file = "../Resources/city_df.csv"
city_data= pd.read_csv(output_data_file)
city_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Khuldābād,20.0500,75.1833,58.73,32,65,6.49,IN,1612740621
1,1,Albany,42.6001,-73.9662,28.99,63,99,3.00,US,1612740304
2,2,Soe,-9.8607,124.2840,72.37,88,100,6.33,ID,1612740623
3,3,Atasū,48.6814,71.6433,33.13,96,100,30.00,KZ,1612740624
4,4,Lebu,-37.6167,-73.6500,60.76,87,54,3.87,CL,1612740396


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)

#Use the Lat and Lng as locations and Humidity as the weight.
locations = city_data[['Lat', 'Lng']]
weights = city_data['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Narrow down the cities to fit weather conditions
perfect_city = city_data.loc[(city_data["Max Temp"] > 70) & (city_data["Max Temp"] < 90) & (city_data["Cloudiness"] == 0), :]
perfect_city = perfect_city.dropna(how='any')
perfect_city.reset_index(inplace=True)
del perfect_city['Unnamed: 0']
del perfect_city['index']
perfect_city.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Buin,-33.7333,-70.7500,75.20,46,0,8.05,CL,1612740647
1,Santa Fe,-31.6333,-60.7000,84.00,51,0,4.00,AR,1612740609
2,Calbuco,-41.7680,-73.1273,73.40,56,0,9.22,CL,1612740731
3,Rio Grande,-32.0350,-52.0986,71.01,73,0,10.18,BR,1612740801
4,Lázaro Cárdenas,17.9583,-102.2000,78.62,67,0,5.73,MX,1612740806


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#hotel map
hotel_df= []
query_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for x in range(len(perfect_city)):
    lat = perfect_city.loc[x]['Lat']
    lng = perfect_city.loc[x]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    response = requests.get(query_url, params=params).json()
    
    try:
        hotel_df.append(response['results'][0]['name'])
    except:
        hotel_df.append("")
perfect_city["Hotel Name"] = hotel_df
perfect_city = perfect_city.dropna(how='any')
perfect_city.head()




,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Buin,-33.7333,-70.7500,75.20,46,0,8.05,CL,1612740647,San Bernardo
1,Santa Fe,-31.6333,-60.7000,84.00,51,0,4.00,AR,1612740609,Santa Fe
2,Calbuco,-41.7680,-73.1273,73.40,56,0,9.22,CL,1612740731,Calbuco
3,Rio Grande,-32.0350,-52.0986,71.01,73,0,10.18,BR,1612740801,Rio Grande
4,Lázaro Cárdenas,17.9583,-102.2000,78.62,67,0,5.73,MX,1612740806,Lázaro Cárdenas


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in perfect_city.iterrows()]
locations = perfect_city[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))